##  Setup

In [1]:
import Pkg
Pkg.add("Flux")
Pkg.add("Zygote")

   Resolving package versions...
      Compat entries added for Flux
    Updating `~/Repos/AWiD/MyMlp/Project.toml`
  [587475ba] + Flux v0.16.4
    Updating `~/Repos/AWiD/MyMlp/Manifest.toml`
  [082447d4] + ChainRules v1.72.4
  [bbf7d656] + CommonSubexpressions v0.3.1
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + DiffRules v1.15.1
  [f151be2c] + EnzymeCore v0.8.11
  [587475ba] + Flux v0.16.4
  [f6369f11] + ForwardDiff v1.0.1
  [d9f16b24] + Functors v0.5.2
  [7869d1d1] + IRTools v0.4.14
  [7e8f7934] + MLDataDevices v1.10.0
  [0b1bfda6] + OneHotArrays v0.2.10
  [3bd65402] + Optimisers v0.4.6
  [33c8b6b6] + ProgressLogging v0.1.4
  [c1ae055f] + RealDot v0.1.0
  [dc90abb0] + SparseInverseSubset v0.1.2
  [09ab397b] + StructArrays v0.7.1
  [e88e6eb3] + Zygote v0.7.9
  [700de1a5] + ZygoteRules v0.2.7
  [05823500] ↓ OpenLibm_jll v0.8.1+4 ⇒ v0.8.1+2
Precompiling project...
    975.7 ms  ✓ libevdev_jll
    987.8 ms  ✓ mtdev_jll
   1018.0 ms  ✓ Xorg_xcb_util_jll
   1123.6 ms  ✓ libinput_jll
  

In [1]:
using Printf
using LinearAlgebra
using Random
using Flux
using Zygote


##  Przygotowanie danych testowych

In [2]:
# Mini słownik (bardzo mały dla prostoty)
vocab = ["good", "bad", "movie", "very", "<PAD>"]
vocab_size = length(vocab)
embedding_dim = 3  # Małe embedding dla łatwych obliczeń
sequence_length = 4  # Długość sekwencji
num_of_words = 5  # Liczba słów w słowniku

println("Słownik:")
for (i, word) in enumerate(vocab)
    println("$i: $word")
end

println("\nParametry:")
println("- Rozmiar słownika: $vocab_size")
println("- Wymiar embedding: $embedding_dim")
println("- Długość sekwencji: $sequence_length")
println("- Liczba słów w słowniku: $num_of_words")

# Sekwencje tekstowe jako indeksy
# Sekwencja 1: "very good movie <PAD>" → [5, 2, 4, 1] → pozytywna (klasa 1)
# Sekwencja 2: "very bad movie <PAD>" → [5, 3, 4, 1] → negatywna (klasa 0)

sequence_1 = Float32[4, 1, 3, 5]  # very good movie <PAD>
sequence_2 = Float32[4, 2, 3, 5]  # very bad movie <PAD>

# Konwertuj na format batch (sequence_length, batch_size)
X_batch = zeros(Float32, sequence_length, 2)
X_batch[:, 1] = sequence_1
X_batch[:, 2] = sequence_2

# Etykiety: sekwencja 1 → pozytywna (1), sekwencja 2 → negatywna (0)
y_batch = Float32[1.0 0.0]  # (1, 2) 

println("Sekwencja 1 (pozytywna): ", [vocab[Int(i)] for i in sequence_1])
println("Indeksy: ", sequence_1)
println("\nSekwencja 2 (negatywna): ", [vocab[Int(i)] for i in sequence_2])
println("Indeksy: ", sequence_2)
println("\nShape danych X: ", size(X_batch))
println("Etykiety y: ", y_batch)
println("Shape etykiet: ", size(y_batch))

# Embedding weights - każde słowo ma swój unikalny wektor
embedding_weights = Float32[
    1.0  -1.0   0.5  0.0 0.0;   # wymiar 1: <PAD>, good, bad, movie, very
    0.5   0.5   0.0  1.0 0.0;   # wymiar 2
    1.0  -1.0   1.0  0.0 0.0    # wymiar 3
]

for (i, word) in enumerate(vocab)
    vec = embedding_weights[:, i]
    println("$word: $vec")
end

Słownik:
1: good
2: bad
3: movie
4: very
5: <PAD>

Parametry:
- Rozmiar słownika: 5
- Wymiar embedding: 3
- Długość sekwencji: 4
- Liczba słów w słowniku: 5
Sekwencja 1 (pozytywna): ["very", "good", "movie", "<PAD>"]
Indeksy: Float32[4.0, 1.0, 3.0, 5.0]

Sekwencja 2 (negatywna): ["very", "bad", "movie", "<PAD>"]
Indeksy: Float32[4.0, 2.0, 3.0, 5.0]

Shape danych X: (4, 2)
Etykiety y: Float32[1.0 0.0]
Shape etykiet: (1, 2)
good: Float32[1.0, 0.5, 1.0]
bad: Float32[-1.0, 0.5, -1.0]
movie: Float32[0.5, 0.0, 1.0]
very: Float32[0.0, 1.0, 0.0]
<PAD>: Float32[0.0, 0.0, 0.0]


##  Warstwa embedding - Flux

In [3]:
# Tworzymy warstwę embedding: 5 tokenów (słownik), każdy reprezentowany przez wektor 3-wymiarowy
embedding_layer = Flux.Embedding(5, 3)

embedding_layer.weight .= embedding_weights  # Ustawiamy wagi embeddingu na nasze predefiniowane wartości

x_batch = Int64.(X_batch)  # Konwersja do Int64, ponieważ Flux.Embedding oczekuje indeksów jako Int

4×2 Matrix{Int64}:
 4  4
 1  2
 3  3
 5  5

In [4]:
# Używamy pullback, aby uzyskać wynik przejścia "forward" i funkcję do propagacji wstecznej
# Chcemy gradienty względem parametrów warstwy, więc musimy je "wyciągnąć" z modelu
params = Flux.params(embedding_layer)
y_output, back = Flux.pullback(() -> embedding_layer(x_batch), params)

println("--- PRZEJŚCIE W PRZÓD ---")
println("Wynik z embedding_layer(X_batch):")
println("Shape: ", size(y_output))

y_output

┌ Warning: `Flux.params(m...)` is deprecated. Use `Flux.trainable(model)` for parameter collection,
│ and the explicit `gradient(m -> loss(m, x, y), model)` for gradient computation.
└ @ Flux /home/oliwier/.julia/packages/Flux/9PibT/src/deprecations.jl:93


--- PRZEJŚCIE W PRZÓD ---
Wynik z embedding_layer(X_batch):
Shape: (3, 4, 2)


3×4×2 Array{Float32, 3}:
[:, :, 1] =
 0.0  1.0  0.5  0.0
 1.0  0.5  0.0  0.0
 0.0  1.0  1.0  0.0

[:, :, 2] =
 0.0  -1.0  0.5  0.0
 1.0   0.5  0.0  0.0
 0.0  -1.0  1.0  0.0

##  Warstwa embedding - Flux Backward

In [5]:
# Załóżmy, że gradient spływający z góry (z kolejnych warstw/funkcji straty)
# to tensor samych jedynek o takim samym kształcie jak wyjście `y_output`.
# W terminologii Zygote, ten gradient nazywa się `Δy` (delta y).
Δy = ones(Float32, size(y_output))

println("\n--- PRZEJŚCIE W TYŁ ---")
println("Gradient spływający z góry (Δy) to same jedynki o kształcie: ", size(Δy))

# Wywołujemy funkcję `back`, aby obliczyć gradienty.
# Przekazujemy jej nasz sfabrykowany gradient `Δy`.
grads = back(Δy)

# `grads` to obiekt typu Zygote.Grads, z którego możemy wyciągnąć gradient dla wag
embedding_weight_gradients = grads[embedding_layer.weight]

println("\nObliczony gradient dla wag embeddingu (embedding_layer.weight):")
println("Shape: ", size(embedding_weight_gradients))

embedding_weight_gradients


--- PRZEJŚCIE W TYŁ ---
Gradient spływający z góry (Δy) to same jedynki o kształcie: (3, 4, 2)

Obliczony gradient dla wag embeddingu (embedding_layer.weight):
Shape: (3, 5)


3×5 Matrix{Float32}:
 1.0  1.0  2.0  2.0  2.0
 1.0  1.0  2.0  2.0  2.0
 1.0  1.0  2.0  2.0  2.0

##  Warstwa Dense 3D Flux - Forward

In [11]:
embedding_dim = 3
sequence_length = 4
batch_size = 2

# Chcemy transformować każdy wektor o długości `embedding_dim`
# na nowy wektor o długości `dense_output_dim`.
dense_output_dim = 2 # Załóżmy, że chcemy uzyskać wektory 5-wymiarowe

# ## ZMIANA: Dodajemy `relu` jako funkcję aktywacji ##
dense_layer = Dense(embedding_dim => dense_output_dim, relu)

# Predefiniowane wagi i bias (te same)
custom_weights = Float32[
    10.0  0.0  -1.0;
     0.0  1.0   2.0
]
custom_bias = Float32[1.0; -2.0]

dense_layer.weight .= custom_weights
dense_layer.bias .= custom_bias

println("\n--- WARSTWA DENSE z ReLU ---")
println("Wagi (W):\n", dense_layer.weight)
println("Bias (b):\n", dense_layer.bias)


--- WARSTWA DENSE z ReLU ---
Wagi (W):
Float32[10.0 0.0 -1.0; 0.0 1.0 2.0]
Bias (b):
Float32[1.0, -2.0]


In [14]:
# Obliczamy najpierw wynik liniowy, przed `relu`
linear_output = zeros(Float32, dense_output_dim, sequence_length, batch_size)
for i in 1:batch_size
    for j in 1:sequence_length
        linear_output[:, j, i] = dense_layer.weight * y_output[:, j, i] + dense_layer.bias
    end
end

params = Flux.params(dense_layer)
Y, back = Flux.pullback(() -> dense_layer(y_output), params)

println("\n--- PRZEJŚCIE W PRZÓD (Wynik) ---")
println("Wynik liniowy (przed ReLU) dla batch_idx=1:\n", linear_output[:, :, 1])
println("\nOstateczny wynik (Y, po ReLU) dla batch_idx=1:\n", Y[:, :, 1])

Y


--- PRZEJŚCIE W PRZÓD (Wynik) ---
Wynik liniowy (przed ReLU) dla batch_idx=1:
Float32[1.0 10.0 5.0 1.0; -1.0 0.5 0.0 -2.0]

Ostateczny wynik (Y, po ReLU) dla batch_idx=1:
Float32[1.0 10.0 5.0 1.0; 0.0 0.5 0.0 0.0]


2×4×2 Array{Float32, 3}:
[:, :, 1] =
 1.0  10.0  5.0  1.0
 0.0   0.5  0.0  0.0

[:, :, 2] =
 1.0  0.0  5.0  1.0
 0.0  0.0  0.0  0.0

##  Dense 3D - Flux Backward

In [15]:
# Gradient spływający z góry - same jedynki
ΔY = ones(Float32, size(Y))

println("\n--- PRZEJŚCIE W TYŁ ---")
println("Gradient spływający z góry (ΔY) to same jedynki o kształcie: ", size(ΔY))

# ## KLUCZOWY MOMENT: Propagacja gradientu przez ReLU ##
# Gradient przepłynie tylko tam, gdzie Y > 0.
# Tworzymy maskę gradientu na podstawie wyniku `Y`.
# ΔY_after_relu będzie naszym efektywnym gradientem po przejściu przez pochodną ReLU.
ΔY_after_relu = ΔY .* (Y .> 0)
println("\nEfektywny gradient po przejściu przez pochodną ReLU (ΔY_after_relu) dla batch_idx=1:\n", ΔY_after_relu[:, :, 1])

# Obliczamy gradienty za pomocą Flux
grads = back(ΔY)

dense_weight_gradients = grads[dense_layer.weight]
dense_bias_gradients = grads[dense_layer.bias]

println("\n--- WYNIKI GRADIENTÓW (Flux) ---")
println("Obliczony gradient dla wag (∂L/∂W):\n", dense_weight_gradients)
println("\nObliczony gradient dla biasu (∂L/∂b):\n", dense_bias_gradients)


--- PRZEJŚCIE W TYŁ ---
Gradient spływający z góry (ΔY) to same jedynki o kształcie: (2, 4, 2)

Efektywny gradient po przejściu przez pochodną ReLU (ΔY_after_relu) dla batch_idx=1:
Float32[1.0 1.0 1.0 1.0; 0.0 1.0 0.0 0.0]

--- WYNIKI GRADIENTÓW (Flux) ---
Obliczony gradient dla wag (∂L/∂W):
Float32[2.0 2.5 3.0; 1.0 0.5 1.0]

Obliczony gradient dla biasu (∂L/∂b):
Float32[7.0, 1.0]


##  Max Pool (4,) Flux

In [21]:

println("--- DANE WEJŚCIOWE DLA WARSTWY POOLING ---")
X = Y

pooling_layer = MaxPool((4,))

X

--- DANE WEJŚCIOWE DLA WARSTWY POOLING ---


2×4×2 Array{Float32, 3}:
[:, :, 1] =
 1.0  10.0  5.0  1.0
 0.0   0.5  0.0  0.0

[:, :, 2] =
 1.0  0.0  5.0  1.0
 0.0  0.0  0.0  0.0

##  Max Pool (4,) Forward - Flux

In [24]:
X_permuted = permutedims(X, (2, 1, 3))  # Przekształcamy wymiary na (sequence_length, batch_size, embedding_dim)

Y, back = Flux.pullback(pooling_layer, X_permuted)

println("\n--- TEST 1: MaxPool((4,)) ---")
println("--- PRZEJŚCIE W PRZÓD (Wynik) ---")
println("Kształt wyniku (Y): ", size(Y)) # Powinno być (2, 1, 2)
println("\nWynik dla batch_idx=1 (po 'wyciśnięciu' wymiaru sekwencji):\n", Y[:, 1, 1])
println("\nWynik dla batch_idx=2 (po 'wyciśnięciu' wymiaru sekwencji):\n", Y[:, 1, 2])
Y


--- TEST 1: MaxPool((4,)) ---
--- PRZEJŚCIE W PRZÓD (Wynik) ---
Kształt wyniku (Y): (1, 2, 2)

Wynik dla batch_idx=1 (po 'wyciśnięciu' wymiaru sekwencji):
Float32[10.0]

Wynik dla batch_idx=2 (po 'wyciśnięciu' wymiaru sekwencji):
Float32[5.0]


1×2×2 Array{Float32, 3}:
[:, :, 1] =
 10.0  0.5

[:, :, 2] =
 5.0  0.0